In [1]:
import pandas as pd
import json
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#pd.set_option('display.max_colwidth', None)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
df = pd.read_json('priority.json')

questions = pd.DataFrame(df.loc['questions']['result']).fillna('')

In [2]:
questions['category'] = df.loc['name']['result']

In [3]:
# theory
from IPython.display import Image, HTML
HTML(json.loads(df.loc['theorie']['result'])['en'].replace('/cdn','https://www.theorieexamen.nl/cdn'))

In [4]:
def en_q(x):
    try:
        return json.loads(x)['en']
    except:
        return str(x).replace('Nee','No').replace('Ja','Yes').replace('nan','')

questions['content'] = questions.text.apply(en_q)

In [5]:
questions.text[0]

'{"nl":"Je rijdt hier. Heeft de fietser voorrang?","en":"Does the cyclist go first?"}'

In [6]:
questions['explaination'] = questions.motivation.apply(en_q)
questions['answer_1'] = questions.answers.apply(pd.Series)[0].apply(pd.Series).text.apply(en_q)
questions['answer_2'] = questions.answers.apply(pd.Series)[1].apply(pd.Series).text.apply(en_q)
questions['answer_3'] = questions.answers.apply(pd.Series)[2].apply(pd.Series).text.apply(en_q)

In [7]:
questions['image_src'] = '<img src="'+ questions['image_url'].str.replace('/cdn','https://www.theorieexamen.nl/cdn')  + '"style="max-width:100%; width: 200%"/>'

In [8]:
#html_show = questions[['id','category_id','category','is_open_question','image_src','image_credits','created_at','updated_at','content','answer_1','answer_2','answer_3','explaination']].fillna('')
html_show = questions[['image_src','content','answer_1','answer_2','answer_3']].fillna('')
html_show = html_show.style.set_properties(subset=['image_src'], **{'width': '300px'})
#questions[['id','content','answer_1','answer_2','answer_3']].fillna('')
HTML(html_show.to_html(escape=False))

,image_src,content,answer_1,answer_2,answer_3
0,,Does the cyclist go first?,No,Yes,
1,,Where do you have to give way to a bus that wants to pull away from a bus stop?,Within the built-up area,Inside and outside the built-up area,Outside the built-up area
2,,Which statement is true?,You may cut across a military convoy if you drive on a priority road,A military convoy may be crossed by a vehicle approaching from the right on a priority uncontrolled junction,A military convoy must never be crossed
3,,Do you have to stop for a red traffic light when you drive in the middle of a military convoy?,Yes,No,
4,,How can you recognise the first vehicle in a military convoy?,It has a blue flag on the left and right and a blue cap on the right headlight,It has a green flag on the left and right and a green cap on the right headlight,It has a green flag on the left and a green cap on the right headlight
5,,Which statement is true?,Trams always go first on a priority uncontrolled junction,Trams always go first,Trams only go first when they approach from the right on a priority uncontrolled junction
6,,Must pedestrians wait for a tram at a pedestrian crossing?,No,Yes,
7,,Who goes first?,The oncoming vehicle,You,
8,,Which statement is true?,The traffic on the side of the road with the obstacle must give way to oncoming traffic,The traffic on the side of the road without the obstacle must give way to oncoming traffic,The traffic that comes last must give way to the traffic that comes first
9,,You want to turn into the road. To whom must you give way?,All road users,Drivers coming from the right,Road users coming from the right


In [153]:
import pandas as pd
import json
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#pd.set_option('display.max_colwidth', None)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import Image, HTML
import os
json_files = [pos_json for pos_json in os.listdir('./') if pos_json.endswith('.json')]


def en_q(x):
    try:
        return json.loads(x)['en']
    except:
        return str(x).replace('Nee','No').replace('Ja','Yes').replace('nan','')

def answer_url(x):
    if '/cdn' in x:
        r = '<img src="'+ x.replace('/cdn','https://www.theorieexamen.nl/cdn')  + '"style="max-width:100%; width: 100%"/>'
        return r
    else:
        return x
    
    
all_questions = pd.DataFrame()
for j in json_files:
    df = pd.read_json(j)
    questions = pd.DataFrame(df.loc['questions']['result']).fillna('')
    questions['category'] = j.replace('.json','').replace('_',' ')#df.loc['name']['result']
    questions['content'] = questions.text.apply(en_q)
    questions['explaination'] = questions.motivation.apply(en_q)
    questions['answer_1'] = questions.answers.apply(pd.Series)[0].apply(pd.Series).text.apply(en_q).apply(answer_url)
    questions['answer_2'] = questions.answers.apply(pd.Series)[1].apply(pd.Series).text.apply(en_q)
    questions['answer_3'] = questions.answers.apply(pd.Series)[2].apply(pd.Series).text.apply(en_q)
    questions['image_src'] = ('<img src="'+ questions['image_url'].str.replace('/cdn','https://www.theorieexamen.nl/cdn')  \
                            + '"style="max-width:100%; width: 200%"/>').replace('<img src=""style="max-width:100%; width: 200%"/>','')
    all_questions = all_questions.append(questions)
    
    
html_show = all_questions.fillna('')[['category','is_open_question','image_src','content','answer_1','answer_2','answer_3','explaination']]

In [12]:
all_questions.to_csv('all_questions.csv')

#HTML(all_questions[['category','image_src','content','updated_at']].sort_values('updated_at', ascending=False).reset_index(drop=True).to_html(escape=False))

In [164]:
select_img = html_show[html_show['answer_1'].str.contains('/cdn/')].reset_index(drop=True)[['category','content','answer_1','answer_2','explaination']]
select_img['answer_2'] = select_img['answer_2'].str.replace('/cdn','<img src="https://www.theorieexamen.nl/cdn')\
                        .str.replace('.png','.png"style="max-width:40%; width: 100%"/>')\
                        .str.replace('.svg','.svg"style="max-width:40%; width: 100%"/>')\
                        .str.replace('.jpg','.jpg"style="max-width:40%; width: 100%"/>')\
                        .str.replace('.gif','.gif"style="max-width:40%; width: 100%"/>')
select_img['correct'] = select_img['answer_1'].str.replace('max-width:100%','max-width:40%')

import random

def shuffle(x):
    random.shuffle(x)
    return x

select_img['answers'] = (select_img['correct'] +','+ select_img['answer_2']).str.split(',').apply(shuffle).str.join('')

select_img = select_img[['category','content','answers','correct','explaination']]



#html_show = html_show.style.set_properties(subset=['image_src'], **{'width': '200px'})
select_img = select_img.style.set_properties(**{'text-align': 'right'})\
                            .set_properties( **{'border-bottom': '2px solid black'})
#questions[['id','content','answer_1','answer_2','answer_3']].fillna('')
#HTML(select_img.to_html(open('select_img.html', 'w'),escape=False, header = False))
HTML(select_img.to_html(escape=False))

,category,content,answers,correct,explaination
0,Distance and Speed,Which sign indicates the end of the maximum speed?,,,The white sign /cdn/b/f8.svg indicates the end of the maximum speed. The square sign indicates the end of the recommended speed. The sign with the round red border indicates a parking ban.
1,Eco Driving,When does the engine use the most fuel?,,,The engine uses more fuel at high revs. Shift up early to a higher gear. Petrol engines should shift between 2000 and 3000 rpm and diesel engines between 1500 and 2500 rpm.
2,Eco Driving,You drive a petrol car. When should you switch gears if you want to save fuel?,,,The engine uses more fuel at high revs. Shift up early to a higher gear.
3,Lighting,Which lights are allowed to be used at all times?,,,The symbol /cdn/o/dashboard/dipped-beam.jpg indicates the dipped beam headlights. You may always use dipped headlights. Dipped beams are compulsory at night and in low visibility conditions during the day.
4,Manoeuvres,On which road can you make a u-turn?,,,"It is not permitted to make a u-turn on a one-way street or autoweg (expressway). You may make a u-turn on a priority road. The yellow-white sign /cdn/b/b1.svg indicates a priority road. The sign with the red border /cdn/b/f7.svg indicates a u-turn ban. The blue sign with the arrow /cdn/b/c3.svg indicates a one-way street. The blue sign with the symbol of a car /cdn/b/g3.svg indicates an autoweg. Making a u-turn is a special manoeuvre, you must give way to all other traffic."
5,Motorway and Autoweg,Which sign indicates that the motorway is going to split in two motorways?,,,"The sign /cdn/b2/bb03.png indicates that the motorway splits into two motorways. The roads on the left and right of the sign are motorways. The traffic sign with the text 'uit' /cdn/b2/bb01.png indicates that you leave the motorway. The sign /cdn/b2/bb04.png indicates an interruption, narrowing or the end of the emergency lane. The sign /cdn/b2/bb11l-scherpe-bocht.png indicates a sharp or dangerous bend."
6,Motorway and Autoweg,Which sign indicates the end of the emergency lane?,,,"The sign /cdn/b2/bb04.png indicates an interruption, narrowing or the end of the emergency lane. The red-white sign /cdn/b2/bb11l-scherpe-bocht.png indicates a sharp or dangerous bend. The green-white sign /cdn/b2/bb03.png indicates that the motorway splits into two motorways. The roads on the left and right of this sign are motorways. The traffic sign with the text 'uit' /cdn/b2/bb01.png indicates that you leave the motorway."
7,Motorway and Autoweg,On which road can you expect crossing traffic?,,,"The sign with the symbol of a car /cdn/b/g3.svg indicates an autoweg (expressway). On an autoweg you may have to deal with intersecting and joining traffic. The other sign /cdn/b/g1.svg indicates a motorway. On a motorway you do not expect crossing traffic, but flyovers/overpasses (such as tunnels and viaducts)."
8,Motorway and Autoweg,Matrix signs are used above lanes. Which lane do you not have to leave?,,,"The matrix sign /cdn/b/matrixbord-einde-sm.png indicates the end of all prohibitions and advice's shown on matrix signs. From now on, the normal traffic rules apply. You don't have to change lanes. The arrow /cdn/b/matrixbord-verdrijfpijl-sm.gif is an initial warning of a red cross. The red cross /cdn/b/matrixbord-kruis-sm.gif indicates the lane is closed."
9,Number and Facts,Which vehicle is the last vehicle of a military convoy?,,,The last vehicle has a green flag on the right and a green cap on the right headlight.


In [8]:
open_questions = html_show[html_show['is_open_question'] == 1].reset_index(drop=True)[['category','image_src','content','answer_1']]

open_questions = open_questions.sort_values(['category','image_src']).reset_index(drop=True)
#open_questions = open_questions.style.set_properties(subset=['image_src'], **{'width': '300px'})
HTML(open_questions.to_html(open('open_questions.html', 'w'),escape=False, header = False))

<IPython.core.display.HTML object>

In [160]:
normal = html_show[~html_show['answer_1'].str.contains('/cdn/') & (html_show['is_open_question'] == 0)].reset_index(drop=True)
normal = normal[['category','image_src','content','answer_1','answer_2','answer_3','explaination']]

def remove_na(l):
    return [x for x in l if str(x) != '']

normal['answers'] = normal[['answer_1','answer_2','answer_3']].agg(';'.join, axis=1).str.split(';').apply(shuffle).apply(remove_na)\
                    .str.join('<br><br>')
normal['correct'] = normal['answer_1']
normal = normal[['category','image_src','content','answers','correct','explaination']]


knowledge = normal[~normal['category'].isin(['Priority Situations','Priority','Hazard Perception','Stopping and Parking'])]
knowledge = knowledge.sort_values(['category','image_src']).reset_index(drop=True)


k_flaged = knowledge[knowledge.index.isin(flaged[flaged['second'].isin(['x','c'])]['flaged'].tolist() )]

knowledge = knowledge.style.set_properties(subset=['image_src'], **{'width': '300px'})\
                            .set_properties( **{'border-bottom': '2px solid black'})

knowledge.to_html(open('knowledge.html', 'w', encoding="utf-8"),escape=False, notebook=True, border=1)


priority = normal[normal['category'].isin(['Priority Situations','Priority','Stopping and Parking'])]
priority = priority.sort_values(['category','image_src']).reset_index(drop=True)
priority = priority.style.set_properties(subset=['image_src'], **{'width': '300px'})\
                            .set_properties( **{'border-bottom': '2px solid black'})
priority.to_html(open('priority.html', 'w', encoding="utf-8"),escape=False)

In [110]:

HTML(normal.to_html(escape=False))

,category,image_src,content,answer_1,answer_2,answer_3,anwsers,correct
0,Accidents and Breakdowns,,Is it mandatory to carry a warning triangle in your car?,No,Yes,,YesNo,No
1,Accidents and Breakdowns,,Is it mandatory to have a first aid kit in your car?,No,Yes,,NoYes,No
2,Accidents and Breakdowns,,Your car breaks down in a tunnel. What should you do?,You try to drive out of the tunnel first and then look for a place to stop,You park the car as far as possible on the right side of the tunnel and turn off the engine,You switch on the main beam headlights and all occupants must get out of the car,You try to drive out of the tunnel first and then look for a place to stopYou park the car as far as possible on the right side of the tunnel and turn off the engineYou switch on the main beam headlights and all occupants must get out of the car,You try to drive out of the tunnel first and then look for a place to stop
3,Accidents and Breakdowns,,Your car breaks down and you can't drive out of the tunnel. What should you do now?,"You park the car as far as possible on the right, switch on the hazard lights and switch off the engine",You park the car as far as possible on the left and turn on the hazard lights,You park the car as far as possible on the right and switch on the hazard lights,"You park the car as far as possible on the left and turn on the hazard lightsYou park the car as far as possible on the right, switch on the hazard lights and switch off the engineYou park the car as far as possible on the right and switch on the hazard lights","You park the car as far as possible on the right, switch on the hazard lights and switch off the engine"
4,Accidents and Breakdowns,,What is the best place to keep an emergency hammer?,Right of the driver's seat,In the dashboard,Left of the driver's seat,Left of the driver's seatRight of the driver's seatIn the dashboard,Right of the driver's seat
5,Accidents and Breakdowns,,You want to tow another car but you only have driving license B. Is that permitted?,Yes,No,,YesNo,Yes
6,Accidents and Breakdowns,,Your car has crashed into the water. It's better to wait until your car is filled with water before leaving the car. Is that correct?,No,Yes,,YesNo,No
7,Accidents and Breakdowns,,Your car has crashed into the water. Which window is the easiest to break?,The side window in a corner,The side window in the middle,The front window in the middle,The front window in the middleThe side window in a cornerThe side window in the middle,The side window in a corner
8,Accidents and Breakdowns,,What percentage of traffic accidents is caused by fatigue?,15 percent,5 percent,25 percent,5 percent15 percent25 percent,15 percent
9,Accidents and Breakdowns,,What do you do first when your vehicle crashes into the water?,Turn on the lights,Open the side window,Unbelt,Open the side windowTurn on the lightsUnbelt,Turn on the lights


In [126]:
priority.sort_values(['category','image_src'])

,category,image_src,content,answer_1,answer_2,answer_3
31,Accidents and Breakdowns,,On which roads is the risk of aquaplaning the ...,At 100 to 130 km / h roads,At 50 km / h roads,At 80 km / h roads
50,Accidents and Breakdowns,"<img src=""https://www.theorieexamen.nl/cdn/aut...",You're waiting for the breakdown service. Are ...,No,Yes,
6,Accidents and Breakdowns,"<img src=""https://www.theorieexamen.nl/cdn/c/o...",Your car has crashed into the water. It's bett...,No,Yes,
10,Accidents and Breakdowns,"<img src=""https://www.theorieexamen.nl/cdn/c/o...",Your car crashes into the water. You immediate...,No,Yes,
33,Accidents and Breakdowns,"<img src=""https://www.theorieexamen.nl/cdn/c/o...",Which lane has more often ruts in the road sur...,Lane C,Lane A,Lane B
...,...,...,...,...,...,...
868,Your Car,"<img src=""https://www.theorieexamen.nl/cdn/ove...",Which area can you see through the wing mirror?,The green area,The red area,The blue area
855,Your Car,"<img src=""https://www.theorieexamen.nl/cdn/ts-...",What do you have to take into account if you c...,The car will tilt more when you make a turn,It will have a positive influence on the road ...,The braking distance will decrease
869,Your Car,"<img src=""https://www.theorieexamen.nl/cdn/ts-...",Your front window is damaged. Are you permitte...,No,Yes,
874,Your Car,"<img src=""https://www.theorieexamen.nl/cdn/weg...",What is the most common cause of a tyre blowout?,An incorrect tyre pressure,Poor road conditions,An overloaded vehicle


In [159]:
flaged = pd.read_csv('flaged.csv',encoding = 'gbk')

In [161]:


HTML(k_flaged.to_html(escape=False))

,category,image_src,content,answers,correct,explaination
23,Accidents and Breakdowns,,On which lane of the motorway is the risk of aquaplaning the greatest?,On the left laneOn the deceleration laneOn the right lane,On the right lane,Heavy road users such as lorries drive a lot on the right lane. Heavy road users form ruts in the road surface. With wet weather water fills up in the ruts. This increases the risk of aquaplaning considerably.
59,Accidents and Breakdowns,,Your car breaks down in a tunnel. What should you do?,You park the car as far as possible on the right side of the tunnel and turn off the engineYou switch on the main beam headlights and all occupants must get out of the carYou try to drive out of the tunnel first and then look for a place to stop,You try to drive out of the tunnel first and then look for a place to stop,"If your car breaks down in a tunnel, you should drive out of the tunnel if reasonably possible. Only if that's really not possible and you have to stop in the tunnel, then park your car as far as possible to the right as possible. And turn on the hazard lights and switch off the engine. Leave the key in the car so the emergency services can move the car if necessary."
62,Accidents and Breakdowns,,Your car broke down. Can you stop here?,YesNo,Yes,"Stopping because your car broke down is not the same as voluntarily leaving your car at a standstill. If your car breaks down and you can no longer drive further, you must quickly find a safe place to stop where you obstruct other traffic as little as possible. This can be, for example, the verge, refuge or just a parking space. If you cannot find a suitable place, go to the right as far as possible. That can also be a bus lane. You may only stop on the bus lane in an emergency."
69,Behaviour,,What should you do?,You should often give way to other road usersYou should take priority when you have priorityYou should try to predict traffic situations,You should try to predict traffic situations,Try to look far ahead and predict traffic situations.
105,Distance and Speed,,Up to where does the speed limit apply?,Until a different maximum speed is indicatedUntil the traffic sign end zone maximum speedUntil the next crossing,Until the traffic sign end zone maximum speed,"The zone with maximum speed applies until the traffic sign ""end zone maximum speed""."
110,Distance and Speed,,Which road is an 80 km / h road?,Road BRoad CRoad A,Road A,"Road A is a 80 km / hour road. Based on the road markings you can divide roads outside the built-up area into 3 different types of roads. An 80 km / h road can be recognised by the double lines in the centre of the road. A 100 km / h road can be recognised by the green filled double lines. Road B is a 100 km / hour road. On a road without a line in the centre of the road, the speed usually limited from 80 km / hour to 60 km / hour. Road C is 60 km / h road. The road markings do not change the speed limit. Traffic signs have priority over road markings. The road markings are intended to provide extra clarity and guidance."
121,Driver,,The tunnel effect is a reduction of the field of view. How can you suffer from that?,When you have consumed alcoholWhen you drive through a long tunnelWhen you drive a long distance through a monotonous landscape,When you have consumed alcohol,"One of the effects that arise after consuming alcohol is a reduction in the field of vision, which is called the tunnel effect. Because the eye muscles weaken due to the narcotic effect of alcohol, you can perceive less left and right."
122,Driver,,"Behind you drives a police car. You see the text: ""STOP POLITIE"". Who must stop?",The driver behind the police carAll drivers behind the police carThe driver in front of the police car,The driver in front of the police car,"You have to stop when a police car is driving behind you and you read the text ""STOP POLITIE"" (stop police) via the mirror."
128,Driver,,What is polder blindness?,A reduction in the fie

In [162]:
k_flaged = k_flaged.style.set_properties(subset=['image_src'], **{'width': '300px'})\
                            .set_properties( **{'border-bottom': '2px solid black'})
k_flaged.to_html(open('k_flaged.html', 'w', encoding="utf-8"),escape=False, notebook=True, border=1)

In [1]:
df.empty

NameError: name 'df' is not defined

In [4]:
import pandas as pd

try:
    print(df.empty)
except Exception as e:
    print(e)
    df = pd.DataFrame()

True
